# Create the grid for Richards 1D

This code allows to create a mesh for 1D PDE problem:
    - domain discretization
    - parameters setting
    - initial condition setting
All output data are stored in a NetCDF file

    -Author: Niccolò Tubini and Riccardo Rigon
    -License: Creative Commons 4.0

In [28]:
%%capture
%run RichardsMeshGenVanGenuchten.py

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from netCDF4 import Dataset

In [29]:
# Standard imports 
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot

# from bokeh.models.widgets import Panel, Tabs
##################### Change #######################
from bokeh.models.layouts import TabPanel as Panel
from bokeh.models.layouts import Tabs
# ################################################

from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.models import BoxSelectTool
from bokeh.models import HoverTool

output_notebook()

Loading BokehJS ...

## Define some variables:
- `folderPath`: path of the fodlder containing the input file (.csv) -> your_path/data/RichardMeshGen_input
- `inFileName`: name of the input file (.csv)


- `outFileName`: name of the output file (.nc) with the entire file path -> your_path/data/Grid_NetCDF/grid.nc
- `outputTitle`: string containing a title for the output, it is saved within the file
- `outputSummary`: string containing a description for the output, it is saved within the file (global metadata)
- `outputDate`: string containing the date of file creation, optionally
- `outputInstitution`: string containing the name of whom created the file, optionally

In [30]:
# folderPath = "C:/Users/Niccolo/OMS/OMS_Project_Richards1D/data/RichardMeshGen_input"
folderPath = "C:/Users/USER/Desktop/WDev/byrnewyne"

# inputFileName = "prova.csv"
inputFileName = "Clay_noPonding_BC.csv"

# outputFileName =  "C:/Users/Niccolo/OMS/OMS_Project_Richards1D/data/Grid_NetCDF/cancella.nc"
outputFileName =  "C:/Users/USER/Desktop/WDev/byrnewyne/matumbaiOutputFolder/Clay_noPonding_BC.nc"

outputTitle = '''
            TEST OUTPUT ----------> MATUMBAI      '''
outputSummary = '''
            Type,eta,N,psi,thetaS,thetaR,Ks,alphaSpecificStorage,betaSpecificStorage,n,alpha,et
            L,0,40,-2,0.5,0.07,0.000023,0,0,1.16,5.88,0
            L,-2,nan,0,nan,nan,nan,nan,nan,nan,nan,nan
'''

# outputDate = '14/02/2019'
from datetime import date
today = date.today()
outputDate = today.strftime("%d/%m/%Y")

outputInstitution = 'Geoframe'

## Define the type of intial condition for water suction $\psi$:
Possible choices are:

- `hydrostatic`
- `constant`
- `linear interpolation`
- `piecewise-hydrostatic`

In [31]:
icType = 'piecewise-hydrostatic'

## Set some plot features

In [32]:
labelSize = 18
titleSize = 22
legendSize = 14
axisTicksSize = 14

lineWidth =0.3

In [33]:
os.chdir(folderPath)

In [34]:
data = pd.read_csv(inputFileName)
#print(data.head())
#print(data.index)
#print(data.columns)
data

,Type,eta,N,psi,thetaS,thetaR,Ks,n,psiD,alphaSpecificStorage,betaSpecificStorage,et
0,L,0,40,-2,0.35,0.07,0.000098,1.5,-0.3496,0,0,0
1,L,-2,-999,0,-999.00,-999.00,-999.000000,-999.0,-999.0000,-999,-999,-999


### This section computes geometrical quantities of the discretized domain
For an explanantion of the data in the above dataFrame, please read first the presentation at 

### From the data in the csv file, the main numpy array are built:
- `eta`: vertical coordinate of control volume centroids, positive upward with origin set at soil surface [m]
- `etaDual`: vertical coordinate of control volume interfaces, positive upward with origin set at soil surface [m]
- `deltaZ`: length of each control volume [m]
- `spaceDelta`: distance between two adjacent control volume. It is used to compute gradients [m]
- `z`: vertical coordinate of control volume centroids, positive upward with origin set at soil column bottom [m]
- `zDual`: vertical coordinate of control volume interfaces, positive upward with origin set at soil column bottom [m]

In [35]:
[eta,etaDual,deltaZ,spaceDelta,z,zDual]=buildData(data)

In [36]:
showMesh(data,z,eta,etaDual,labelSize,titleSize,legendSize,axisTicksSize,lineWidth)

### From the data in the csv file, the initial condition for water suction $\psi$ is defined
- hydrostatic distribution for a given $\psi$ @bottom -> 'hydrostatic'
- constant distribution for a given $\psi$ @bottom -> 'constant'
- linear interpolation from measurement points -> 'linear interpolation'

In [37]:
psiIC = setInitialCondition(data,eta,z,icType)

In [38]:
showInitialCondition(psiIC,eta,icType,labelSize,titleSize,legendSize,axisTicksSize,lineWidth)

In [39]:
showTotalHead(psiIC,z,eta,icType,labelSize,titleSize,legendSize,axisTicksSize,lineWidth)

### Assign hydraulic properties to each control volume of the grid
For an explanantion of the data in the above dataFrame, please read first the presentation at 

In [40]:
[thetaS, thetaR, Ks, alphaSpecificStorage, betaSpecificStorage, n, alpha, par3SWRC, par4SWRC, par5SWRC, psiStar, par7SWRC, par8SWRC, et] = setParameters(data,eta)

KeyError: 'alpha'

In [ ]:
showParameters(thetaS,thetaR,Ks,alphaSpecificStorage,betaSpecificStorage,n,alpha,psiStar,et,eta,labelSize,titleSize,axisTicksSize,lineWidth)


## Save all grid data in a NetCDF file


In [ ]:
writeGridNetCDF(eta,etaDual,z,zDual,spaceDelta,deltaZ,psiIC,thetaS,thetaR,Ks,alphaSpecificStorage,betaSpecificStorage,n,alpha,par3SWRC,par4SWRC,par5SWRC,psiStar,par7SWRC,par8SWRC,et,outputFileName,outputTitle,outputInstitution,outputSummary,folderPath,outputDate,inputFileName)

*** SUCCESS writing!  C:/Users/Niccolo/OMS/OMS_Project_Richards1D/data/Grid_NetCDF/Clay_noPonding_VG.nc
